In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/api-dataset/api_dataset/train.json
/kaggle/input/api-dataset/api_dataset/test.json
/kaggle/input/api-dataset/api_dataset/val.json


installing dependencies

In [2]:
!pip install -q transformers datasets evaluate rouge_score nltk


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip install -U transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 101.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 24.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.4.1
    Uninstalling datasets-4.4.1:
      Successfully uninstalled datasets-4.4.1


model configs

In [4]:
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import numpy as np
import nltk

# --- 1. CONFIGURATION ---
MODEL_CHECKPOINT = "t5-base"
BATCH_SIZE = 8  # T5-base fits 8-16 on a T4 GPU
MAX_INPUT_LENGTH = 128   # Metadata usually short
MAX_TARGET_LENGTH = 256  # Descriptions can be longer
LEARNING_RATE = 3e-4
EPOCHS = 1


2025-12-22 22:12:32.524620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766441552.937654      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766441553.047277      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766441554.137576      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766441554.137618      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766441554.137621      55 computation_placer.cc:177] computation placer alr

In [5]:
# --- 2. LOAD DATASET ---
data_files = {
    "train": "/kaggle/input/api-dataset/api_dataset/train.json",
    "validation": "/kaggle/input/api-dataset/api_dataset/val.json",
    "test": "/kaggle/input/api-dataset/api_dataset/test.json"
}

def load_json_dataset():
    train_df = pd.read_json("/kaggle/input/api-dataset/api_dataset/train.json", lines=True)
    val_df = pd.read_json("/kaggle/input/api-dataset/api_dataset/val.json", lines=True)
    test_df = pd.read_json("/kaggle/input/api-dataset/api_dataset/test.json", lines=True)
    
    return DatasetDict({
        "train": Dataset.from_pandas(train_df),
        "validation": Dataset.from_pandas(val_df),
        "test": Dataset.from_pandas(test_df)
    })

raw_datasets = load_json_dataset()
print(f"✅ Data Loaded. Train size: {len(raw_datasets['train'])}")


✅ Data Loaded. Train size: 6341


tokensiser and Preprocessing

In [6]:
# --- 3. PREPROCESSING (Tokenization) ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def preprocess_function(examples):
    inputs = ["document: " + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True
    )

    labels = tokenizer(
        text_target=examples["target_text"],
        max_length=MAX_TARGET_LENGTH,
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

apply tokenization

In [7]:
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True
)


Map:   0%|          | 0/6341 [00:00<?, ? examples/s]

Map:   0%|          | 0/793 [00:00<?, ? examples/s]

Map:   0%|          | 0/793 [00:00<?, ? examples/s]

Evaluation Metrics (BLEU & ROUGE)

In [8]:
import numpy as np
import nltk
import evaluate

# Load metrics once to avoid reloading overhead
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # 1. Handle Tuple Outputs
    if isinstance(predictions, tuple):
        predictions = predictions[0]
        
    # 2. Sanitize Predictions & Labels (-100 -> Pad ID)
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # 3. Decode Tokens -> Text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # 4. Post-process for ROUGE (Newline after sentences)
    decoded_preds_clean = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels_clean = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # 5. Compute ROUGE
    # use_stemmer=True matches standard paper implementations (like gDoc)
    result = rouge_metric.compute(predictions=decoded_preds_clean, references=decoded_labels_clean, use_stemmer=True)
    
    # 6. Compute BLEU
    # BLEU expects references as a list of lists: [['ref1'], ['ref2']]
    # Safety: Replace empty strings with "empty" to prevent crashes
    bleu_refs = [[l if l.strip() else "empty"] for l in decoded_labels]
    bleu_preds = [p if p.strip() else "empty" for p in decoded_preds]
    
    bleu_score = bleu_metric.compute(predictions=bleu_preds, references=bleu_refs)
    
    # Add BLEU to results
    result["bleu"] = bleu_score["bleu"]
    
    # 7. Scale to percentages (0.35 -> 35.0) and round
    return {k: round(v * 100, 4) for k, v in result.items()}


FileNotFoundError: Couldn't find a module script at /kaggle/working/bleu/bleu.py. Module 'bleu' doesn't exist on the Hugging Face Hub either.

Model & Training Arguments

In [ ]:
import transformers
print(transformers.__version__)


In [ ]:
# --- 5. MODEL & TRAINER ---
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

args = Seq2SeqTrainingArguments(
    output_dir=f"{MODEL_CHECKPOINT}-finetuned-openapi",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=True,
    num_train_epochs=EPOCHS,
    warmup_steps=200,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model
)


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


# --- 6. TRAIN ---
print("🚀 Starting T5 Fine-tuning...")
trainer.train()


In [ ]:
# --- 7. SAVE MODEL ---
trainer.save_model("my_t5_api_model")
print("✅ Model Saved!")

In [15]:
# --- 1. Imports & Setup ---
import numpy as np
import nltk
import evaluate
from transformers import (
    AutoModelForSeq2SeqLM, 
    AutoTokenizer, 
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments, 
    DataCollatorForSeq2Seq
)

# Ensure NLTK data is available
nltk.download("punkt")

# --- 2. Load Metric Objects ---
print("⏳ Loading metrics (ROUGE, BLEU, BERTScore)...")
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

# --- 3. Load Model & Tokenizer ---
# UPDATE THIS PATH to your exact checkpoint folder
MODEL_PATH = "/kaggle/working/t5-base-finetuned-openapi/checkpoint-397" 

print(f"🔄 Loading model from: {MODEL_PATH}")
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# --- 4. Define Compute Metrics Function ---
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if isinstance(predictions, tuple): 
        predictions = predictions[0]
    
    # 1. Replace -100 (ignore index) with pad token
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # 2. Decode to text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # 3. Clean Text (Remove extra whitespace)
    decoded_preds_clean = [pred.strip() for pred in decoded_preds]
    decoded_labels_clean = [label.strip() for label in decoded_labels]

    # 4. Prepare for ROUGE (Sentence Splitting is standard for ROUGE)
    decoded_preds_rouge = ["\n".join(nltk.sent_tokenize(pred)) for pred in decoded_preds_clean]
    decoded_labels_rouge = ["\n".join(nltk.sent_tokenize(label)) for label in decoded_labels_clean]
    
    # --- CALCULATION ---
    
    # A. ROUGE (Lexical Overlap)
    result = rouge.compute(
        predictions=decoded_preds_rouge, 
        references=decoded_labels_rouge, 
        use_stemmer=True
    )
    
    # B. BLEU (Precision) - Use Clean Text
    # BLEU expects references to be a list of lists: [['ref1'], ['ref2']]
    bleu_refs = [[l] for l in decoded_labels_clean]
    bleu_score = bleu.compute(predictions=decoded_preds_clean, references=bleu_refs)
    result["bleu"] = bleu_score["bleu"]
    
    # C. BERTScore (Semantic Similarity) - Use Clean Text
    # lang="en" downloads the correct roberta-large model automatically
    bert_result = bertscore.compute(
        predictions=decoded_preds_clean, 
        references=decoded_labels_clean, 
        lang="en"
    )
    # We take the mean F1 score of the batch
    result["bertscore_f1"] = np.mean(bert_result["f1"])
    
    # Round metrics for display
    final_metrics = {k: round(v * 100, 4) for k, v in result.items()}
    return final_metrics

# --- 5. Setup Trainer for Evaluation ---
args = Seq2SeqTrainingArguments(
    output_dir="t5_final_eval_results",
    per_device_eval_batch_size=16,  # Lower if you run out of VRAM (BERTScore is heavy)
    predict_with_generate=True,
    generation_max_length=128,
    fp16=True,                     # Speed up inference
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Use the 'test' dataset you defined earlier
# Ensure 'tokenized_datasets' is loaded in your environment!
if 'tokenized_datasets' not in globals():
    raise ValueError("⚠️ Please load your 'tokenized_datasets' before running this script!")

tester = Seq2SeqTrainer(
    model=model,
    args=args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# --- 6. Run & Print Results ---
print("🚀 Running FINAL Benchmark on T5 (ROUGE + BLEU + BERTScore)...")
# This might take 3-5 minutes depending on test set size
metrics = tester.evaluate(eval_dataset=tokenized_datasets["test"])

print("\n" + "="*40)
print("🏆 T5-BASE FINAL RESULTS")
print("="*40)
print(f"ROUGE-1:      {metrics['eval_rouge1']:.2f}")
print(f"ROUGE-2:      {metrics['eval_rouge2']:.2f}")
print(f"ROUGE-L:      {metrics['eval_rougeL']:.2f}")
print("-" * 40)
print(f"BLEU:         {metrics['eval_bleu']:.2f}")
print("-" * 40)
print(f"BERTScore F1: {metrics['eval_bertscore_f1']:.2f}")
print("="*40)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


⏳ Loading metrics (ROUGE, BLEU, BERTScore)...
🔄 Loading model from: /kaggle/working/t5-base-finetuned-openapi/checkpoint-397


/tmp/ipykernel_55/42003414.py:98: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  tester = Seq2SeqTrainer(


🚀 Running FINAL Benchmark on T5 (ROUGE + BLEU + BERTScore)...


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🏆 T5-BASE FINAL RESULTS
ROUGE-1:      41.54
ROUGE-2:      26.91
ROUGE-L:      39.20
----------------------------------------
BLEU:         2.77
----------------------------------------
BERTScore F1: 88.97


In [ ]:
pip install bert_score

In [ ]:
# --- 6. RUN EVALUATION + EXTRACT EXAMPLES ---
print("🚀 Running FINAL Benchmark on T5...")
metrics = tester.evaluate(eval_dataset=tokenized_datasets["test"])

# --- 7. FIXED: Generate 3 Examples for Report ---
print("\n🔍 Generating 3 Example Predictions...")

# Fix the label_ids by replacing -100 with pad_token_id BEFORE decoding
test_subset = tokenized_datasets["test"].select(range(3))
test_results = tester.predict(test_subset)

# 🔥 FIX: Clean predictions and labels
predictions_clean = np.where(test_results.predictions != -100, test_results.predictions, tokenizer.pad_token_id)
labels_clean = np.where(test_results.label_ids != -100, test_results.label_ids, tokenizer.pad_token_id)

# Decode examples
example_preds = tokenizer.batch_decode(predictions_clean, skip_special_tokens=True)
example_refs = tokenizer.batch_decode(labels_clean, skip_special_tokens=True)
example_inputs = tokenizer.batch_decode(test_subset["input_ids"], skip_special_tokens=True)

# --- 8. DISPLAY RESULTS ---
print("\n" + "="*50)
print("🏆 T5-BASE FINAL RESULTS")
print("="*50)
print(f"ROUGE-1:      {metrics['eval_rouge1']:.2f}")
print(f"ROUGE-2:      {metrics['eval_rouge2']:.2f}")
print(f"ROUGE-L:      {metrics['eval_rougeL']:.2f}")
print("-" * 30)
print(f"BLEU:         {metrics['eval_bleu']:.2f}")
print("-" * 30)
print(f"BERTScore F1: {metrics['eval_bertscore_f1']:.2f}")
print("="*50)

print("\n" + "="*70)
print("📋 EXAMPLE PREDICTIONS (First 3 Test Samples)")
print("="*70)
for i in range(3):
    print(f"\n--- EXAMPLE {i+1} ---")
    print(f"INPUT:   {example_inputs[i]}")
    print(f"PRED:    {example_preds[i].strip()}")
    print(f"REF:     {example_refs[i].strip()}")
    print("-" * 50)
